# 使用教學  

> <font size=4> 1. 請先和系辦曾小姐索取今年申請本系轉系、輔系或雙主修的學生成績檔案</font>   
> <font size=4> 2. 若成績檔的檔案類型為.xls檔，請將其另存為.xlsx檔</font>    
> <font size=4> 3. 點擊左上方的「」圖示並按下左上方的「」按鍵將這些新的xlsx檔上傳</font>    
> <font size=4> 4. 到"設定變數"下的程式碼當中尋找變數grade_path，分別設定申請轉系、輔系或雙主修學生的成績檔的名稱</font>    
> <font size=4> 5. 按下上方的「」按鍵執行以下3組程式碼</font>    
> <font size=4> 6. 等程式跑完以後，在左方的檔案區中找到名稱含有"results"的xlsx檔案後，點右鍵下載至電腦</font>    
> <font size=4> 7. 嘗試手動計算2~3位學生的平均成績並與結果檔案中記錄的平均成績進行比對</font>    
> <font size=4> 8. 若無問題即可將結果寄給系辦曾小姐</font>  
>     
> <font size=3> 若有任何問題請聯繫作者施彥任(F08524004@ntu.edu.tw)</font>    
> <font size=3> 最後編輯時間：2023/7/12</font>  

# 載入模組

In [1]:
from ntuche_tmdm import arrangement

# 設定變數

In [2]:
"""
Parameters
----------
core_course1 : list, str
    本校所有微積分、普通化學與普通物理學課名的共通字串
grade_path: list, str
    存放申請輔系、轉系與雙主修學生的成績檔案儲存路徑
"""
core_course1= ['微積分', '普通化學', '普通物理學']
grade_path = ['111輔系.xlsx', '111轉系.xlsx', '111雙主修.xlsx']

# 執行計算並儲存為excel檔

In [3]:
"""
Parameters
----------
savepathi: str
    結果檔案的儲存路徑
studentrank: object of class "arrangement"
    執行所有計算的物件
sheetnamei: str
    結果檔案的工作表名稱
"""

for grade_pathi in grade_path:
    grade_path1, grade_path2 = grade_pathi.split('.')
    savepathi = grade_path1 + '_results.' + grade_path2
    studentranki = arrangement(grade_pathi, core_course1)
    sheetnamei = 'results'
    studentranki.save_df_data(studentranki.df_alldata, savepathi, sheetnamei)